# Exercise 4.8:

Directions

Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.





In [2]:
# import libraries:
import pandas as pd
import numpy as np
import os

In [3]:
# import data
path = r'/Users/azadehrezaei/Desktop/Career Foundry/Achievement 4/08-2021 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_newvariables.pkl'))

## Question 2:
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Question 3:
Analyze the result. How do the results for the entire dataframe differ from those of the subset? 

- The results of this aggregation on the entire dataframe compared to the subset of 1000,000 rows shows very similar values. This probably shows that a subset of a million rows can be considered a suitable sample dataset for this project. 
Some of the values are slightly different, for example department_id 3's mean in the entire dataframe is 17/170395 whereas it is 16.084944. The maximum mean value in the sample set is department_id 21, which is 25.535479 but the same departments mean value is 22.902379 (still the highest, but now much closer to the second highest value which is 20.197148).



## Question 4:
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [5]:
# create a new column called 'max_order, apply the .groupby() function on the user_id and then the .transform() 
#function is applied on the 'order_number' with the np.max argument, saving the result in the new max_order column.
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

- create a 'loyalty_flag' column:

In [6]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [10]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(20)

,user_id,loyalty_flag,order_number
0,138,Regular customer,28
1,138,Regular customer,30
2,709,New customer,2
3,764,New customer,1
4,764,New customer,3
5,777,Regular customer,16
6,825,New customer,3
7,910,Regular customer,12
8,1052,Regular customer,10
9,1052,Regular customer,15


## Question 5:
The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

- To compare the spending habits of these three groups I decided to group the dataframe by the 'loyalty_flag' column and use .agg to find the basic statistics of these three groups:
The results actually show that the new customers have a higher average value followed by regular customers and then loyalty customers. this is probably because we determined the flag values by the frequency of the purchase rather than the dollar amount.  

In [11]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'median']})

prices                     
                       mean  min      max median
loyalty_flag                                    
Loyal customer    10.386336  1.0  99999.0    7.4
New customer      13.294670  1.0  99999.0    7.4
Regular customer  12.495717  1.0  99999.0    7.4

## Question 6:
The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
  If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
  If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
  
- I started by creating a new column in my dataframe labeled 'avg_prices' to store the average prices grouped by the user_id. 

In [12]:
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,slowest/busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,Mid-range Product,Regularly busy,Regular Busy,Most Orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,Mid-range Product,Regularly busy,Regular Busy,Average Orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,Mid-range Product,Busiest day,Buesiest Day,Average Orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,Mid-range Product,Regularly busy,Least Busy,Most Orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,Mid-range Product,Least busy,Least Busy,Average Orders,3,New customer,4.972414


- I used the avg_prices column to create the flag column labeled 'spending_flag' based on the criteria in the question. 

In [15]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low Spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High Spender'

In [18]:
ords_prods_merge['spending_flag'].value_counts(dropna = True)

Low Spender     31770746
High Spender      634113
Name: spending_flag, dtype: int64

## Question 7:
In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


- I created a new column in the datafram labeled 'order_freq' and used .groupby() to group the rows by their user_id and then applied transform on the 'days_since_prior_order' column with the np.median argument to get the median number of days it takes each user before placing the next order.  

In [19]:
ords_prods_merge['order_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,_merge,price_range_loc,busiest_day,slowest/busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,order_freq
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range Product,Regularly busy,Regular Busy,Most Orders,32,Regular customer,6.935811,Low Spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range Product,Regularly busy,Regular Busy,Average Orders,32,Regular customer,6.935811,Low Spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range Product,Busiest day,Buesiest Day,Average Orders,5,New customer,7.930208,Low Spender,6.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range Product,Regularly busy,Least Busy,Most Orders,3,New customer,4.972414,Low Spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range Product,Least busy,Least Busy,Average Orders,3,New customer,4.972414,Low Spender,9.0


- I used .loc() to create a flag column based on the results of the 'order_freq' column. I labeled the new flag column 'frequency_flag':

In [21]:
ords_prods_merge.loc[ords_prods_merge['order_freq'] > 20, 'frequency_flag'] = 'Non-frequent Customer'

In [22]:
ords_prods_merge.loc[(ords_prods_merge['order_freq'] > 10) & (ords_prods_merge['order_freq'] <= 20) , 'frequency_flag'] = 'Regular Customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['order_freq'] < 10, 'frequency_flag'] = 'Frequent Customer'

In [27]:
ords_prods_merge['frequency_flag'].value_counts(dropna = True)

Frequent Customer        21491229
Regular Customer          6921016
Non-frequent Customer     2686728
Name: frequency_flag, dtype: int64

In [26]:
# Export data:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.8_ords_prods_merge.pkl'))